In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import re
from numpy import where as IF

# set the graphs to show in the jupyter notebook
%matplotlib inline

# set seabor graphs to a better style
sns.set(style="ticks")

In [ ]:
Acqs=pd.read_csv('../input/credit-card-exploratory-data-analysis/Customer Acqusition.csv')
spend=pd.read_csv('../input/credit-card-exploratory-data-analysis/spend.csv')
repayment=pd.read_csv('../input/credit-card-exploratory-data-analysis/Repayment.csv')

In [ ]:
data=Acqs.merge(spend,how='outer',on='Customer').merge(repayment,how='outer',on=['Customer','Month'])

In [ ]:
del data['SL No:']
del data['Sl No:']
del data['No']

In [ ]:
data.shape

1. In the above dataset,
a. In case age is less than 18, replace it with mean of age values.

In [ ]:
avg=round(Acqs.Age.mean(),2)

In [ ]:
Acqs.Age[Acqs.Age<18]=avg

b. In case spend amount is more than the limit, replace it with 50% of that customer’s limit.
(customer’s limit provided in acquisition table is the per transaction limit on his card)

In [ ]:
data.Amount_x[data.Amount_x>data.Limit]=.5*data.Limit

c. Incase the repayment amount is more than the limit, replace the repayment with the
limit.

In [ ]:
data.Amount_y[data.Amount_y>data.Limit]=data.Limit

2. From the above dataset create the following summaries:

a. How many distinct customers exist?

In [ ]:
data.Customer.loc[-data.Customer.duplicated()].count()

 b. How many distinct categories exist?

In [ ]:
a=[]
a.append(data.Type.loc[-data.Type.duplicated()].count())
a.append(data.City.loc[-data.City.duplicated()].count())
a.append(data.Product.loc[-data.Product.duplicated()].count())
a.append(data.Company.loc[-data.Company.duplicated()].count())
a.append(data.Segment.loc[-data.Segment.duplicated()].count())
sum(a)

c. What is the average monthly spend by customers?

In [ ]:
spend.groupby(['Month'])['Amount'].mean().reset_index()

d. What is the average monthly repayment by customers?

In [ ]:
repayment.groupby(['Month'])['Amount'].mean().reset_index()

e. If the monthly rate of interest is 2.9%, what is the profit for the bank for each month?
(Profit is defined as interest earned on Monthly Profit. Monthly Profit = Monthly repayment
– Monthly spend. Interest is earned only on positive profits and not on negative amounts)

In [ ]:
data['profit']=data.Amount_y-data.Amount_x

In [ ]:
data['pos_profit']=np.where(data.profit<0,0,data.profit)*.029

In [ ]:
data.groupby(['Month'])[['pos_profit']].sum().reset_index()

f. What are the top 5 product types?

In [ ]:
data.groupby(['Type'])[['profit']].sum().sort_values(by='profit',ascending=False).reset_index().head(5)

g. Which city is having maximum spend?

In [ ]:
data.groupby(['City'])[['Amount_x']].sum().sort_values(by='Amount_x').reset_index().head(1)

 h. Which age group is spending more money?

In [ ]:
data['Age_Group'] = IF((data.Age>0) & (data.Age<=10),"Teenager",
   IF((data.Age>=11) & (data.Age<=30),"Young",
      IF((data.Age>=31) & (data.Age<=55),"Middle",
         IF((data.Age>=56) & (data.Age<=80),"Senior","Old"))))

In [ ]:
data.groupby(['Age_Group'])[['Amount_x']].sum().sort_values(by='Amount_x',ascending=False).head(1).reset_index()

i. Who are the top 10 customers in terms of repayment?

In [ ]:
data.groupby(['Customer'])[['Amount_y']].sum().sort_values(by='Amount_y',ascending=False).reset_index().head(10)

3. Calculate the city wise spend on each product on yearly basis. Also include a graphical
representation for the same.

In [ ]:
data.Month=data.Month.astype('datetime64')

In [ ]:
data['year']=data.Month.dt.year

In [ ]:
graph1=data.groupby([data.City,data.Product,data.year])[['Amount_x']].sum().reset_index()

In [ ]:
customer_spend_pivot = pd.pivot_table(data = graph1,index=["City","year"],
                                      columns='Product',values="Amount_x")
customer_spend_pivot.plot(kind="bar",figsize=(18,5))
plt.show()

4. Create graphs for

 a. Monthly comparison of total spends, city wise

In [ ]:
data['Monthly']= data['Month'].dt.strftime('%B')

In [ ]:
graph2=data.groupby(['City','Monthly'])[['Amount_x']].sum().sort_index().reset_index()

In [ ]:
graph2_wide=graph2.pivot_table(index='City',columns='Monthly',values='Amount_x')

In [ ]:
graph2_wide.plot(kind='bar',figsize=(18,5))
plt.show()

 b. Comparison of yearly spend on air tickets

In [ ]:
graph3=data.groupby(['year',data.Type[data.Type=='AIR TICKET']])[['Amount_x']].sum().reset_index()

In [ ]:
graph3_wide=graph3.pivot_table(index='year',columns='Type',values='Amount_x')

In [ ]:
graph3_wide.plot(kind='bar',legend=False)
plt.show()

 c. Comparison of monthly spend for each product (look for any seasonality
 that exists in terms of spend)

In [ ]:
graph4=data.groupby(['Monthly','Product'])[['Amount_x']].sum()
graph4_wide=graph4.pivot_table(index='Product',columns='Monthly',values='Amount_x')

In [ ]:
graph4_wide.plot(kind='bar',figsize=(18,6))
plt.show()

5. Write user defined PYTHON function to perform the following analysis:
You need to find top 10 customers for each city in terms of their repayment amount by
different products and by different time periods i.e. year or month. The user should be able
to specify the product (Gold/Silver/Platinum) and time period (yearly or monthly) and the
function should automatically take these inputs while identifying the top 10 customers.


In [ ]:
def udf(x,y):
    return data.groupby(['Customer','City',data.Product[data.Product==x],data.year[data.year==y]])[['Amount_y']].sum().sort_values(by='Amount_y',ascending=False).reset_index().head(10)

In [ ]:
udf(input(),int(input()))